## 라이브러리 로드

In [1]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져옵니다.
import requests
# BeautifulSoup 을 통해 읽어 온 웹페이지를 파싱합니다.
from bs4 import BeautifulSoup as bs
# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 불러옵니다.
import pandas as pd

## 서울시 코로나19 발생동향
* [코로나19](http://www.seoul.go.kr/coronaV/coronaStatus.do)

## requests 로 html 문서 받아오기

In [2]:
# 크롤링 할 사이트
base_url = "http://www.seoul.go.kr/coronaV/coronaStatus.do?menu_code=01"
response = requests.get( base_url )
response

<Response [200]>

In [3]:
if response.status_code == 200:
    soup = bs(response.text, 'html.parser')

str(soup)[:1000]

'\n<!DOCTYPE HTML>\n\n<html lang="ko">\n<head>\n<meta charset="utf-8"/>\n<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, initial-scale=1.0" name="viewport">\n<title>서울시 코로나19</title>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 일일브리핑, 공정 공급마스크&amp;홍보물, 보도자료, 소식지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인 문화생활, 팩트체크, 심리지원단, 정책 제안하기), 시민 참여(잠시멈춤 캠페인, 온서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." name="description"/>\n<meta content="website" property="og:type"/>\n<meta content="서울시 코로나19 통합사이트" property="og:title"/>\n<meta content="서울시 코로나19 통합정보 사이트로 안전·방역(발생동향, 클린존, 선별진료소, 일일브리핑, 공정 공급마스크&amp;홍보물, 보도자료, 소식지, 신고(응답소)), 생활정보(주요뉴스, 생활경제 지원, 온라인 문화생활, 팩트체크, 심리지원단, 정책 제안하기), 시민 참여(잠시멈춤 캠페인, 온서울캠페인, 시민제안)등의 정보를 제공하고 있습니다." property="og:description"/>\n<meta content="https://www.seoul.go.kr/coronaV/coronaStatus.do" property="og:url"/>\n<meta content="http://www.seoul.go.kr/res_newseoul/images/corona/pic_facebook_20200325.jpg" property="og:image"/>\n<meta con

## 서울 확진자 현황

In [4]:
#move-cont1 > div:nth-child(2) > div
cont_page = soup.select("#move-cont1 > div.status-confirm > div.cont-page-wrap > div.cont-page")
len(cont_page)

7

In [5]:
trs_ex =  cont_page[0].select("tr")
trs_ex[:3]

[<tr> <th scope="col">연번</th> <th scope="col">환자</th> <th scope="col">확진일</th> <th scope="col">거주지</th> <th scope="col">여행력</th> <th scope="col">접촉력</th> <th scope="col">조치사항</th> </tr>,
 <tr> <th scope="row">621</th> <td data-tit="환자 번호" scope="col">10635</td> <td data-tit="확진일">4.16.</td> <td data-tit="거주지">용산구</td> <td data-tit="여행력">미국</td> <td data-tit="접촉력">해외 접촉 추정</td> <td data-tit="조치사항">보라매병원</td> </tr>,
 <tr> <th scope="row">620</th> <td data-tit="환자 번호" scope="col">10632</td> <td data-tit="확진일">4.16.</td> <td data-tit="거주지">강남구</td> <td data-tit="여행력">-</td> <td data-tit="접촉력">#10054 접촉</td> <td data-tit="조치사항">서울의료원</td> </tr>]

## 행 데이터 리스트로 만들기

In [6]:
def parse_tr(trs):
    table = []
    for tr in trs:
        tds = tr.select("td")
        if len(tds) > 0 :
            row = []
            # 연번
            number = tr.select("th")[0].get_text()
            row.append(number)
            # 연번 외 데이터
            for td in tds:
                val = td.get_text()
                row.append(val)
            table.append(row)
    return table

parse_tr(trs_ex[:3])

[['621', '10635', '4.16.', '용산구', '미국', '해외 접촉 추정', '보라매병원'],
 ['620', '10632', '4.16.', '강남구', '-', '#10054 접촉', '서울의료원']]

## 페이지마다 가져오기

In [7]:
#move-cont1 > div:nth-child(2) > table
#move-cont1 > div:nth-child(2) > table.tstyle05.tstyleP > thead > tr > th:nth-child(1)
#cont-page2
table_result = []
for page in cont_page:
    trs = page.select("tr")
    result = parse_tr(trs)
    table_result.extend(result)

table_result[:5]

[['621', '10635', '4.16.', '용산구', '미국', '해외 접촉 추정', '보라매병원'],
 ['620', '10632', '4.16.', '강남구', '-', '#10054 접촉', '서울의료원'],
 ['619', '10597', '4.15.', '서대문구', '미국', '해외 접촉 추정', '적십자병원'],
 ['618', '10610', '4.14.', '중구', '-', '#9800 접촉', '보라매병원'],
 ['617', '10589', '4.14.', '서초구', '스페인', '해외 접촉 추정', '보라매병원']]

## 컬럼명 만들기

In [8]:
# 컬럼명 만들기
cols = trs_ex[0].select("th")
cols

[<th scope="col">연번</th>,
 <th scope="col">환자</th>,
 <th scope="col">확진일</th>,
 <th scope="col">거주지</th>,
 <th scope="col">여행력</th>,
 <th scope="col">접촉력</th>,
 <th scope="col">조치사항</th>]

In [9]:
col_name = []
for col in cols:
    col_name.append(col.get_text())
col_name

['연번', '환자', '확진일', '거주지', '여행력', '접촉력', '조치사항']

## 데이터프레임으로 만들기

In [10]:
df = pd.DataFrame(table_result, columns=col_name)
print(df.shape)
df.head()

(621, 7)


,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,621,10635,4.16.,용산구,미국,해외 접촉 추정,보라매병원
1,620,10632,4.16.,강남구,-,#10054 접촉,서울의료원
2,619,10597,4.15.,서대문구,미국,해외 접촉 추정,적십자병원
3,618,10610,4.14.,중구,-,#9800 접촉,보라매병원
4,617,10589,4.14.,서초구,스페인,해외 접촉 추정,보라매병원


## CSV 파일로 저장하기

In [11]:
df.to_csv("covid-19-seoul.csv", index=False)

## CSV 파일 읽어오기

In [12]:
pd.read_csv("covid-19-seoul.csv").head(10)

,연번,환자,확진일,거주지,여행력,접촉력,조치사항
0,621,10635,4.16.,용산구,미국,해외 접촉 추정,보라매병원
1,620,10632,4.16.,강남구,-,#10054 접촉,서울의료원
2,619,10597,4.15.,서대문구,미국,해외 접촉 추정,적십자병원
3,618,10610,4.14.,중구,-,#9800 접촉,보라매병원
4,617,10589,4.14.,서초구,스페인,해외 접촉 추정,보라매병원
5,616,10584,4.14.,서초구,미국,해외 접촉 추정,적십자병원
6,615,10569,4.14.,동작구,영국,해외 접촉 추정,서울의료원
7,614,10575,4.14.,성동구,아일랜드,해외 접촉 추정,보라매병원
8,613,10588,4.13.,성북구,-,#10044 접촉,적십자병원
9,612,10559,4.13.,성동구,미국,해외 접촉 추정,적십자병원


In [13]:
df["접촉력"].value_counts()

해외접촉 추정         183
구로구 콜센터 관련       60
확인중              45
구로구 교회 관련        41
콜센터직원 접촉         35
               ... 
#8044 접촉          1
#9912 접촉          1
#794 접촉자(추정)      1
#8881 접촉 추정       1
#56 접촉자           1
Name: 접촉력, Length: 100, dtype: int64

In [14]:
df["조치사항"].value_counts()

서울의료원           117
보라매병원            89
서울의료원(퇴원)        66
서남병원(퇴원)         64
생활치료센터           51
보라매병원(퇴원)        46
서남병원             37
서북병원(퇴원)         16
국립중앙의료원(퇴원)      15
국립중앙의료원          11
서울대학교병원(퇴원)      10
타시도이관(퇴원)         9
중앙대학교병원(퇴원)       8
생활치료센터(퇴원)        8
서북병원              6
타시도 이관            6
적십자병원             5
타시도 이관(퇴원)        5
서울성모병원            5
상계백병원(퇴원)         4
한일병원(퇴원)          4
은평성모병원            4
서울아산병원            3
강북삼성병원            2
순천향대학병원           2
고대구로병원(퇴원)        2
삼육서울병원(퇴원)        2
순천향서울병원(퇴원)       2
한양대학교병원(퇴원)       1
고대안암병원(퇴원)        1
삼육서울병원            1
경희대학교병원           1
원자력병원             1
세브란스병원(사망)        1
서울대학교병원           1
건대병원              1
서울백병원(퇴원)         1
세브란스병원(퇴원)        1
한양대병원             1
상계병원              1
중앙대병원(퇴원)         1
국립중앙의료원(재입원)      1
서울아산병원(퇴원)        1
삼성서울병원            1
타시도이관             1
서울의료원(사망)         1
고대구로병원            1
강남세브란스병원(퇴원)      1
강남세브란스병원          1
상계백병원             1
